In [ ]:
import os
import json
import pandas as pd
import numpy as np


In [ ]:
# Chemin complet vers le csv servant de source.
source_csv = '../data/metrics/raw_merge_metrics_dataset.csv'

# Chemin de stockage des csv
path = "../data/metrics/"

## <center>**Import du Dataset originel**<center>

In [ ]:
# Création Dataframe
df = pd.read_csv(source_csv, index_col=0).reset_index(drop=True).dropna(axis=1, how="all")

In [ ]:
df.info()
df.head()

## <center>**Préparation du Dataset**<center>

### I - Imputation des données manquantes et suppression des colonnes inutiles 

In [ ]:
# création d'une liste de colonnes à suppr
drop_col = ["id", "message_events"]

In [ ]:
# Renommage de la colonne "timestamp_events"
if "timestamp_events" in df :
    df = df.rename(columns={"timestamp_events" : "timestamp"})

In [ ]:
# Remplacement les données manquants dans la colonne "timestamp_events" par les timestamp de la colonne "created_at"
df.timestamp = df.timestamp.fillna(df["created_at"])

In [ ]:
df.info()

In [ ]:
# Remplacement les données manquants dans la colonne "timestamp_events" par les timestamp de la colonne "created_at"
df.criticality_events = df.criticality_events.fillna(df["status"]).replace("ERR", "ERROR")

In [ ]:
df.criticality_events.unique()

In [ ]:
# ajout des colonnes redondante à la liste de colonnes a suppr
if "created_at" not in drop_col :
    drop_col.append("created_at")
if "name_modules" not in drop_col :
    drop_col.append("name_modules")
if "generation_modules" not in drop_col :
    drop_col.append("generation_modules")
if "status" not in drop_col :
    drop_col.append("status")

print(drop_col)


In [ ]:
df = df.drop(drop_col, axis=1)

In [ ]:
df.head()

In [ ]:
# Vérification de la présence de NaN dans la colonne "identification_events"
df["identification_events"].isnull().values.any()

In [ ]:
df.identification_events = df.identification_events.replace(np.nan, 200)

### II - Modification des données textuelles en données numériques et encodage

In [ ]:
df.info()

#### 1 - Colonne "timestamp"

In [ ]:
# transformation en datetime
df.timestamp = pd.to_datetime(df.timestamp, utc=True)

#### 2 - Colonne "identification_events"

In [ ]:
df.identification_events.unique()

In [ ]:
# Encodage des 5 id textuelles
events_id = []
str_code_dict = {}
str_code = 500
for id in list(df['identification_events'].unique()) :
    try:
        events_id.append(int(id))
    except ValueError:
        str_code_dict[id] = str_code
        events_id.append(str_code)
        str_code += 1
str_code_dict

In [ ]:
# on remplace dans le dataframe les valeurs des id textuelles par leur code correspondant
df.identification_events = df.identification_events.replace(str_code_dict)
# on converti toutes les valeurs en int64
df.identification_events = pd.to_numeric(df.identification_events).astype('int64')

In [ ]:
df.identification_events.unique()

In [ ]:
df.info()

#### 3 - Colonne "varnishLevelsTargetvolume"

In [ ]:
# on converti les float en entier 64
df.varnishLevelsTargetvolume = pd.to_numeric(df.varnishLevelsTargetvolume).astype('int64')

In [ ]:
df.info()

### III - Sauvegarde du nouveau dataframe pour réutilisation

In [ ]:
df.to_csv("../data/metrics/Arthur_dataset_V2.csv")